In [26]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data


def logsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data


def padListOnRightToLengthK(listo,k):
    length=len(listo)
    padded=listo
   
    if length<k:
        for j in [length..k-1]:
            padded=padded+[0]
           
        return padded
    return listo

def constantTF(listo):
    first=listo[0]
    ln=len(listo)
    for j in [0..ln-2]:
        if listo[j]!=listo[j+1]:return False
    return True

def push(listo):
    pushed=[]
    lenth=len(listo)
    pushed=pushed+[listo[lenth-1]]
    for j in [0..lenth-2]:
        pushed=pushed+[listo[j]]
    return pushed

def kIteratesofPush(listo,k):
    if k == 0:return listo
    answer=listo
    for k in [1..k]:
        answer=push(answer)
    return answer
        
def f5(seq, idfun=None): # https://www.peterbe.com/plog/uniqifiers-benchmark
    if idfun is None:
        def idfun(x): return x
        seen = {}
        result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result

def drop_duplicates(seq):return f5(seq,idfun=None)

def zeroLogsOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+[i]
    return answer

def zeroLogsOverFieldDigits(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            di=i.digits(prime)
            answer=answer+[di]
    return answer



In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

In [3]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

201
[0, 3/8*x^2 + 1/2]


In [29]:
def testCandidateZeroLogOverField(f,prime,power,candidate):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    a=t^candidate
    expr3=sage_eval(expr,locals={'x':a})
    if expr3 == 0:tf=True
    return tf


    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=1
bound=120
nomatch=[]
for k in [5..20]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        llm=len(lm)
        print
        print "index: ",index
        tfs=[]
        for k in [0..llm-1]:
            log=lm[k][0]
            candidate=prime*log
            tf=testCandidateZeroLogOverField(poly,prime,sd,candidate)
            tfs=tfs+[tf]
        print tfs
            
            


index:  6
[True, True, True, True, True, True, True, True]

index:  11
[True, True, True, True, True, True, True, True]

index:  16
[True, True, True, True, True, True, True, True]


In [30]:
def testCandidateZeroLogOverField(f,prime,power,candidate):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    a=t^candidate
    expr3=sage_eval(expr,locals={'x':a})
    if expr3 == 0:tf=True
    return tf


    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=1
bound=120
nomatch=[]
for k in [5..20]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        llm=len(lm)
        print
        print "index: ",index
        tfs=[]
        for k in [0..llm-1]:
            log=lm[k][0]
            candidate=prime*log-1
            tf=testCandidateZeroLogOverField(poly,prime,sd,candidate)
            tfs=tfs+[tf]
        print tfs


index:  6
[False, False, False, False, False, False, False, False]

index:  11
[False, False, False, False, False, False, False, False]

index:  16
[False, False, False, False, False, False, False, False]


In [31]:
def testCandidateZeroLogOverField(f,prime,power,candidate):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    a=t^candidate
    expr3=sage_eval(expr,locals={'x':a})
    if expr3 == 0:tf=True
    return tf


    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=0
bound=120
nomatch=[]
for k in [5..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        llm=len(lm)
        print
        print "index: ",index
        tfs=[]
        for k in [0..llm-1]:
            log=lm[k][0]
            candidate=prime*log
            tf=testCandidateZeroLogOverField(poly,prime,sd,candidate)
            tfs=tfs+[tf]
        print tfs
            
            


index:  5
[True, True, True, True]

index:  10
[True, True, True, True]

index:  15
[True, True, True, True]

index:  20
[True, True, True, True]

index:  25
[True, True, True, True]

index:  30
[True, True, True, True]

index:  35
[True, True, True, True]

index:  40
[True, True, True, True]

index:  45
[True, True, True, True]

index:  50
[True, True, True, True]

index:  55
[True, True, True, True]

index:  60
[True, True, True, True]

index:  65
[True, True, True, True]

index:  70
[True, True, True, True]

index:  75
[True, True, True, True]

index:  80
[True, True, True, True]

index:  85
[True, True, True, True]

index:  90
[True, True, True, True]

index:  95
[True, True, True, True]

index:  100
[True, True, True, True]

index:  105
[True, True, True, True]

index:  110
[True, True, True, True]

index:  115
[True, True, True, True]

index:  120
[True, True, True, True]

index:  125
[True, True, True, True]

index:  130
[True, True, True, True]

index:  135
[True, True, True, 

In [32]:
def testCandidateZeroLogOverField(f,prime,power,candidate):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    a=t^candidate
    expr3=sage_eval(expr,locals={'x':a})
    if expr3 == 0:tf=True
    return tf


    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=1
bound=120
nomatch=[]
for k in [5..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        llm=len(lm)
        print
        print "index: ",index
        tfs=[]
        for k in [0..llm-1]:
            log=lm[k][0]
            candidate=prime*log
            tf=testCandidateZeroLogOverField(poly,prime,sd,candidate)
            tfs=tfs+[tf]
        print tfs


index:  6
[True, True, True, True, True, True, True, True]

index:  11
[True, True, True, True, True, True, True, True]

index:  16
[True, True, True, True, True, True, True, True]

index:  21
[True, True, True, True, True, True, True, True]

index:  26
[True, True, True, True, True, True, True, True]

index:  31
[True, True, True, True, True, True, True, True]

index:  36
[True, True, True, True, True, True, True, True]

index:  41
[True, True, True, True, True, True, True, True]

index:  46
[True, True, True, True, True, True, True, True]

index:  51
[True, True, True, True, True, True, True, True]

index:  56
[True, True, True, True, True, True, True, True]

index:  61
[True, True, True, True, True, True, True, True]

index:  66
[True, True, True, True, True, True, True, True]

index:  71
[True, True, True, True, True, True, True, True]

index:  76
[True, True, True, True, True, True, True, True]

index:  81
[True, True, True, True, True, True, True, True]

index:  86
[True, True, 

In [33]:
def testCandidateZeroLogOverField(f,prime,power,candidate):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    a=t^candidate
    expr3=sage_eval(expr,locals={'x':a})
    if expr3 == 0:tf=True
    return tf


    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=2
bound=120
nomatch=[]
for k in [5..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        llm=len(lm)
        print
        print "index: ",index
        tfs=[]
        for k in [0..llm-1]:
            log=lm[k][0]
            candidate=prime*log
            tf=testCandidateZeroLogOverField(poly,prime,sd,candidate)
            tfs=tfs+[tf]
        print tfs


index:  7
[True, True, True, True, True, True, True, True]

index:  12
[True, True, True, True, True, True, True, True]

index:  17
[True, True, True, True, True, True, True, True]

index:  22
[True, True, True, True, True, True, True, True]

index:  27
[True, True, True, True, True, True, True, True]

index:  32
[True, True, True, True, True, True, True, True]

index:  37
[True, True, True, True, True, True, True, True]

index:  42
[True, True, True, True, True, True, True, True]

index:  47
[True, True, True, True, True, True, True, True]

index:  52
[True, True, True, True, True, True, True, True]

index:  57
[True, True, True, True, True, True, True, True]

index:  62
[True, True, True, True, True, True, True, True]

index:  67
[True, True, True, True, True, True, True, True]

index:  72
[True, True, True, True, True, True, True, True]

index:  77
[True, True, True, True, True, True, True, True]

index:  82
[True, True, True, True, True, True, True, True]

index:  87
[True, True, 

In [34]:
def testCandidateZeroLogOverField(f,prime,power,candidate):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    a=t^candidate
    expr3=sage_eval(expr,locals={'x':a})
    if expr3 == 0:tf=True
    return tf


    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=3
bound=120
nomatch=[]
for k in [5..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        llm=len(lm)
        print
        print "index: ",index
        tfs=[]
        for k in [0..llm-1]:
            log=lm[k][0]
            candidate=prime*log
            tf=testCandidateZeroLogOverField(poly,prime,sd,candidate)
            tfs=tfs+[tf]
        print tfs


index:  8
[True, True, True, True, True, True, True, True, True, True]

index:  13
[True, True, True, True, True, True, True, True, True, True]

index:  18
[True, True, True, True, True, True, True, True, True, True]

index:  23
[True, True, True, True, True, True, True, True, True, True]

index:  28
[True, True, True, True, True, True, True, True, True, True]

index:  33
[True, True, True, True, True, True, True, True, True, True]

index:  38
[True, True, True, True, True, True, True, True, True, True]

index:  43
[True, True, True, True, True, True, True, True, True, True]

index:  48
[True, True, True, True, True, True, True, True, True, True]

index:  53
[True, True, True, True, True, True, True, True, True, True]

index:  58
[True, True, True, True, True, True, True, True, True, True]

index:  63
[True, True, True, True, True, True, True, True, True, True]

index:  68
[True, True, True, True, True, True, True, True, True, True]

index:  73
[True, True, True, True, True, True, Tru

In [35]:
def testCandidateZeroLogOverField(f,prime,power,candidate):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    a=t^candidate
    expr3=sage_eval(expr,locals={'x':a})
    if expr3 == 0:tf=True
    return tf


    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=4
bound=120
nomatch=[]
for k in [5..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        llm=len(lm)
        print
        print "index: ",index
        tfs=[]
        for k in [0..llm-1]:
            log=lm[k][0]
            candidate=prime*log
            tf=testCandidateZeroLogOverField(poly,prime,sd,candidate)
            tfs=tfs+[tf]
        print tfs


index:  4
[True, True, True, True]

index:  9
[True, True, True, True]

index:  14
[True, True, True, True]

index:  19
[True, True, True, True]

index:  24
[True, True, True, True]

index:  29
[True, True, True, True]

index:  34
[True, True, True, True]

index:  39
[True, True, True, True]

index:  44
[True, True, True, True]

index:  49
[True, True, True, True]

index:  54
[True, True, True, True]

index:  59
[True, True, True, True]

index:  64
[True, True, True, True]

index:  69
[True, True, True, True]

index:  74
[True, True, True, True]

index:  79
[True, True, True, True]

index:  84
[True, True, True, True]

index:  89
[True, True, True, True]

index:  94
[True, True, True, True]

index:  99
[True, True, True, True]

index:  104
[True, True, True, True]

index:  109
[True, True, True, True]

index:  114
[True, True, True, True]

index:  119
[True, True, True, True]

index:  124
[True, True, True, True]

index:  129
[True, True, True, True]

index:  134
[True, True, True, Tr